# Constant Elasticity of Substitution Planer Welfare Subsidies Optimizer Over Quantile/Individual Groups

There is some production function or other functional relationship between x and y. When x shifts, y shifts. We are interested in some function over the vector of y. We shift x to maximize some objective over the vector of y. 

$$
\max_{\left\{S_i\right\}_{i \in \left\{1,...,N\right\}}}
O \left(
    \left\{ 
    Y_{it}(X_{it}, S_i) 
    \right\}^{i \in \left\{1,...,N\right\}}
    _{t \in \left\{1,...,T\right\}}
 \right)
$$

Specifically: there is a finite amount of subsidies, there are N individuals observed between month 0 and 24. What are the optimal nutritional subsidies to provide to the N individuals given some function that relates nutritional input to outcome (height), ignoring behavior responses.

The N individuals could be N groups of individuals. There could be many more than N people that are in N groups. For example, groups determined by if someone has below or above median initial height.

## Program Structure

The program below has these ingredients. It is not written in the most generic way possible, but wrritten to work with a CES planer's optimization problem where the planer is optimizing over subsidies for sub-groups of individuals. 

1. Given a dataset, generate subsidy groups within
    - each group could have one individual or could have multiple individuals
    - dependency: [Generate Joint Quantiles from Multiple Continuous Variables as a Categorical Variable with Linear Index](https://fanwangecon.github.io/R4Econ/generate/quantile/VarCateIdxVarsQuantiles.html)
        * group linear index 
        * group label
2. Show Basic Summary Statistics for Each group
    - dependency: [All Numeric Variables Mean + SD + N by Groups](https://github.com/FanWangEcon/R4Econ/blob/master/summarize/summ/ByGroupSumm.R)
3. Various Prediction Function
    - based on some estimation structure that generates overall relationship between intputs and outputs and also individual specific coefficients, some type of production function for example
    - prediction functions that generates new outputs based on subsidies for each prediction function model
    - re-write this every time. 
4. Estimation Function Wrapper
    - default parameters and key shifting parameters as inputs
    - allows for using lapply to estimate for example for multiple parameters
    - invokes optim function, and process results with function parameters
5. Inner Estimation Function
    - what optim invokes
    - estimands is first parameter
    - additional parameters as list
6. Estimation Depencies
    - for constrained to unconstrained transformation
    - for generating fractions
    - for ces objective function

## Program

### Dependencies

To achieve points 1 and 2 and 6 above.

In [157]:
library(tidyverse)
# Point 1
source('C:/Users/fan/R4Econ/summarize/summ/ByGroupSumm.R')
# Point 2
source('C:/Users/fan/R4Econ/generate/quantile/VarCateIdxVarsQuantiles.R')
# Point 6
# get f_frac_asymp_vec()
source('C:/Users/fan/R4Econ/optimization/support/fraction.R')
# get f_planer_obj()
source('C:/Users/fan/R4Econ/optimization/planer/ces/cesplanerobj.R')

### Estimation Objective Function

In the function below:
1. x is the unconstrained transformed fraction from (optimization/support/fraction.R)
2. param.ces is the ces parameter, not that this parameter does not enter the f.subsidy.y.str function
3. f.subsidy.y.str is the name of the estimation prediction function (Step 3) in string 
4. list.subsidy.y.params.other contains a list of parameters needed for f.subsidy.y.str in addition to the 

In [138]:
# Single Parameter Optimization Functions
obj_planer <- function(x, param.ces, f.subsidy.y.str, list.subsidy.y.params.other) {

    # Input list
    # Convert from Estimation x to Actual Fraction between 0 and 1
    list.subsidy.y.params.maximand <- list(vec.subsidy.frac = f_frac_asymp_vec(x))
    list.subsidy.y.params <- append(list.subsidy.y.params.other, list.subsidy.y.params.maximand)
    
    # Call Function
    df.y.subsidized <- do.call(f.subsidy.y.str, list.subsidy.y.params)
    
    # C:\Users\fan\R4Econ\optimization\planer\ces\cesplanerobj.R
    obj <- (-1)*f_planer_obj(vec.y=df.y.subsidized$y_subsidy, param.ces=param.ces)
    
    return(obj)
}

### Estimation Wrapper

Basically same parameters as above, except that the first parameter here are the starting values for estimation. 

1. sca.subsidy.frac.init: unconstrained fraction subsidy transformed x initial starting estimation values
2. param.ces:
3. f.subsidy.y.str: name of the estimation prediction function (Step 3) in string 
4. list.subsidy.y.params.other: contains a list of parameters needed for f.subsidy.y.str.

In [139]:
# Optimization Wrapper
# sca.subsidy.frac.init.default <- numeric((sca.subsidy.groups-1))+1
# Optimization Function
optim_wrapper <- function(sca.subsidy.frac.init, param.ces, f.subsidy.y.str, list.subsidy.y.params.other) {
            
    # Optimization
    res.opti <- optim(sca.subsidy.frac.init, obj_planer, 
                      param.ces = param.ces, 
                      f.subsidy.y.str = f.subsidy.y.str,
                      list.subsidy.y.params.other = list.subsidy.y.params.other)
    
    # Generate Named LIst
    sca.subsidy.lengthm1 = length(sca.subsidy.frac.init)
    list.sca.subsidy.frac.init <- setNames(sca.subsidy.frac.init, 
                                           paste0('sca.subsidy.frac.init.v', 1:sca.subsidy.lengthm1))
    list.par <- setNames(res.opti$par, paste0('par.v', 1:sca.subsidy.lengthm1))
    list.par.frac <- setNames(f_frac_asymp_vec(res.opti$par), paste0('par.frac.v', 1:(sca.subsidy.lengthm1+1)))

    # Collect Results
    list.esti.res <- list(param.ces = param.ces, 
                          subsidy.total = list.subsidy.y.params.other[['subsidy.total']], 
                          par.frac.sum = sum(f_frac_asymp_vec(res.opti$par)),
                          value = res.opti$value,
                          counts.function = (res.opti$counts)[['function']],
                          counts.gradient = (res.opti$counts)[['gradient']],
                          convergence = res.opti$convergence) 
    
    list.esti.res <- append(list.esti.res, list.sca.subsidy.frac.init)
    list.esti.res <- append(list.esti.res, list.par)
    list.esti.res <- append(list.esti.res, list.par.frac)
    # Return
    return(list.esti.res)
}

## Data

### Load Data

In [140]:
# Library
library(tidyverse)
source('C:/Users/fan/HeightProfile/r/tools/preamble.R')

# Load Sample Data
setwd('C:/Users/fan/R4Econ/_data/')
df <- read_csv('height_weight.csv')

Parsed with column specification:
cols(
  S.country = col_character(),
  vil.id = col_double(),
  indi.id = col_double(),
  sex = col_character(),
  svymthRound = col_double(),
  momEdu = col_double(),
  wealthIdx = col_double(),
  hgt = col_double(),
  wgt = col_double(),
  hgt0 = col_double(),
  wgt0 = col_double(),
  prot = col_double(),
  cal = col_double(),
  p.A.prot = col_double(),
  p.A.nProt = col_double()
)


### Data Selection

We will have several data selection rules to generate grouped dataframe for subsidies:

1. Dataframe with two groups: below and above median initial height, each group has multiple individuals
    + implicitly, this means the planer is doing policy conditional on observable characteristics, integrating over empirical conditional distribution of unobservable.
2. Dataframe with 3 individuals: subsidy specific to each individual
    + implicitly, this would only be relevant if we know the individual type. 
3. Dataframe with 4 groups: above and below median initial height, and above below median initial weight as well.

Note for the grouped policy, we need to distinguish between individual subsidy and total group subsidy. We can adjust either, will adjust group level subsidy, then need to interpret results then as fraction of subsidy per person. Graph carefully. 

#### Data 1: df.i4xy

In [141]:
# These are the same as in: https://fanwangecon.github.io/HeightProfile/r/planer/paneloptigN.html
# Select 3 individuals, information from second year, four variables
var.qjnt.grp.idx.i4xy <- 'grp.idx.i4xy'
vars.subsidy.group <- 'i'
df.i4xy <- df %>% filter(svymthRound <= 24 & svymthRound >= 0 & 
              S.country == 'Cebu' & (indi.id == 4 | indi.id == 13  | indi.id == 15)) %>% 
              select(i = indi.id, t = svymthRound, y = hgt, x =prot) %>% 
              mutate(!!var.qjnt.grp.idx.i4xy := group_indices(., !!!syms(vars.subsidy.group))) %>%
              drop_na(i, t, y, x)

# Obtain dataframe with quantile cut
df.i4xy.grps <- tibble(id=c(4,13,15), !!var.qjnt.grp.idx.i4xy := c(1,2,3))
sca.subsidy.groups.i4xy <- 3
# Results
print(paste0('dim(df.i4xy)=', dim(df.i4xy)[1], ',', dim(df.i4xy)[2]))
options(repr.matrix.max.rows=5, repr.matrix.max.cols=20)
df.i4xy

[1] "dim(df.i4xy)=39,5"


i,t,y,x,grp.idx.i4xy
4,0,48.4,0.5,1
4,2,53.9,1.0,1
4,4,59.8,0.5,1
...,...,...,...,...
15,22,84,8.9,3
15,24,85,20.3,3


#### Data 2: df.h0 Cebu

Generate quantile cuts. Generate two separate dataframes. 
Dataframes could bedifferent because some observations are dropped due to not having observation for the quantile by variables. 

In [142]:
# Base Data
df.cebu.t0t24 <- df %>% filter(svymthRound <= 24 & svymthRound >= 0 & 
                               S.country == 'Cebu' & 
                               indi.id <= 60 ) %>% 
            select(i = indi.id, t = svymthRound, y = hgt, x = prot, h0 = hgt0, w = wealthIdx) %>%
            drop_na(i, t, y, x, h0, w)

# Cut Quantiles
var.qjnt.grp.idx.h0 <- 'cebu.grp.idx.h0'
list.cts2quantile.h0 <- list(list(vars=c('h0'), prob=c(0, 0.5, 1.0)))
results.h0 <- df_cut_by_sliced_quantiles_joint(df.cebu.t0t24, var.qjnt.grp.idx.h0, list.cts2quantile.h0,
                                               vars.group_by = c('i'), vars.arrange = c('i', 't'),
                                               drop.any.quantile.na = TRUE, toprint = FALSE)
# Obtain dataframe with quantile cut
df.h0 <- results.h0$df.with.cut.quant
df.h0.grps <- results.h0$df.group.slice1.cnt.mean
sca.subsidy.groups.h0 <- dim(df.h0.grps)[1]

# Show Stats
print(paste0('dim(df.h0)=', dim(df.h0)[1], ',', dim(df.h0)[2]))
df.h0.grps
print(paste0('sca.subsidy.groups.h0=', sca.subsidy.groups.h0))

[1] "dim(df.h0)=749,8"


h0_Qs0e1n2,cebu.grp.idx.h0,mean,n
"[44.2,49.4]; (1) of Qs0e1n2",1,47.99677,31
"(49.4,54]; (2) of Qs0e1n2",2,50.99655,29


[1] "sca.subsidy.groups.h0=2"


#### Data 3: df.h0wlt Guatemala

Two Variables Quantile Cut Guatemala.

In [143]:
# Base Data Women Guatemala
df.guat.t0t24 <- df %>% filter(svymthRound <= 24 & svymthRound >= 0 & 
                               S.country == 'Guatemala') %>% 
            select(i = indi.id, t = svymthRound, y = hgt, x = prot, h0 = hgt0, w = wealthIdx) %>%
            drop_na(i, t, y, x, h0, w)

# Cut Quantiles
var.qjnt.grp.idx.h0wlt <- 'guat.grp.idx.h0wlt'
list.cts2quantile.h0wlt <- list(list(vars=c('h0'), prob=c(0, 0.5, 1.0)), 
                          list(vars=c('w'), prob=c(0, 0.5, 1.0)))
results.h0wlt <- df_cut_by_sliced_quantiles_joint(df.guat.t0t24, var.qjnt.grp.idx.h0wlt, list.cts2quantile.h0wlt,
                                                  vars.group_by = c('i'), vars.arrange = c('i', 't'),
                                                  drop.any.quantile.na = TRUE, toprint = FALSE)
# Obtain dataframe with quantile cut
df.h0wlt <- results.h0wlt$df.with.cut.quant
df.h0wlt.grps <- results.h0wlt$df.group.slice1.cnt.mean
sca.subsidy.groups.h0wlt <- dim(df.h0wlt.grps)[1]

# Show Stats
print(paste0('dim(df.h0wlt)=', dim(df.h0wlt)[1], ',', dim(df.h0wlt)[2]))
df.h0wlt.grps
print(paste0('sca.subsidy.groups.h0wlt=', sca.subsidy.groups.h0wlt))

Joining, by = c("i", "t", "y", "x", "h0", "w")


[1] "dim(df.h0wlt)=1808,9"


h0_Qs0e1n2,w_Qs0e1n2,guat.grp.idx.h0wlt,h0_mean,w_mean,h0_n,w_n
"[40.6,49.9]; (1) of Qs0e1n2","[1,2.2]; (1) of Qs0e1n2",1,47.64340,1.725472,106,106
"[40.6,49.9]; (1) of Qs0e1n2","(2.2,5.7]; (2) of Qs0e1n2",2,48.33924,3.153165,79,79
"(49.9,56.7]; (2) of Qs0e1n2","[1,2.2]; (1) of Qs0e1n2",3,51.45867,1.654667,75,75
"(49.9,56.7]; (2) of Qs0e1n2","(2.2,5.7]; (2) of Qs0e1n2",4,51.72400,3.090000,100,100


[1] "sca.subsidy.groups.h0wlt=4"


## Estimation and Prediction Functions

Run Estimations Separately for different Data group, and generate different prediction functions based on them. 

Only three fixed effects for df.i4xy, about 60 for df.h0, and then about 360 for df.h0wlt. 

First run a timer.

In [144]:
# Regressions
f_res_linfe <- function(df) lm(log(y) ~ log(x) + factor(i) - 1 , data=df)

In [145]:
# Timing
system.time({f_res_linfe(df.i4xy)})
system.time({f_res_linfe(df.h0)})
system.time({f_res_linfe(df.h0wlt)})

   user  system elapsed 
   0.02    0.00    0.02 

   user  system elapsed 
   0.01    0.00    0.02 

   user  system elapsed 
   0.24    0.00    0.24 

In [146]:
# StoreEstimation Results
res.linfe.i4xy <- lm(log(y) ~ log(x) + factor(i) - 1 , data=df.i4xy)
res.linfe.h0 <- lm(log(y) ~ log(x) + factor(i) - 1 , data=df.h0)
res.linfe.h0wlt <- lm(log(y) ~ log(x) + factor(i) - 1 , data=df.h0wlt)

In [147]:
# Summaries
# str(res.linfe.h0)
# summary(res.linfe.h0)
# summary(res.linfe.h0wlt)
# summary(res.linfe.i4xy)

In [148]:
f_subsidy_y_cd <- function(df, res.linfe, var.grp.idx, subsidy.total, vec.subsidy.frac) {
    
    # Invoke Function 1
    df.wth.subsidy <- df %>% mutate(subsidy_grp = paste0(vec.subsidy.frac, collapse=','), 
                                    subsidy = subsidy.total*vec.subsidy.frac[df[[var.grp.idx]]])
    
    # Y with subsidy for linear regresion model
    df.wth.subsidy %>% mutate(y_subsidy =
                              exp(predict(res.linfe,
                                          (df.wth.subsidy %>% 
                                           mutate(x = x + subsidy)))))
}

## Planer Optimization Parameters Set Up

### For 3 Individuals

In [149]:
# These are the same as in: https://fanwangecon.github.io/HeightProfile/r/planer/paneloptigN.html
# Should produce the same results
# Default Parameters for Estimation Set up
f.subsidy.y.str.i4xy <- 'f_subsidy_y_cd'
t.filter.default.i4xy <- 24
subsidy.total.default.i4xy <- 100
# lists of Inputs
df.default.i4xy <- df.i4xy %>% filter(t == t.filter.default.i4xy)
list.subsidy.y.params.other.default.i4xy <- list(df = df.default.i4xy, 
                                            res.linfe = res.linfe.i4xy,
                                            var.grp.idx = var.qjnt.grp.idx.i4xy,
                                            subsidy.total = subsidy.total.default.i4xy)
list.subsidy.y.params.maximand.default.i4xy <- list(vec.subsidy.frac =
                                               round(f_subsidy_frac(sca.subsidy.groups.i4xy, 'rand', seed=13), 3))
# Overall
list.subsidy.y.params.i4xy <- append(list.subsidy.y.params.other.default.i4xy,
                                     list.subsidy.y.params.maximand.default.i4xy)

### For h0

In [150]:
# Default Parameters for Estimation Set up
f.subsidy.y.str.h0 <- 'f_subsidy_y_cd'
t.filter.default.h0 <- 24
subsidy.total.default.h0 <- 100

df.default.h0 <- df.h0 %>% filter(t == t.filter.default.h0)

# lists of Inputs
list.subsidy.y.params.other.default.h0 <- list(df = df.default.h0, 
                                               res.linfe = res.linfe.h0,
                                               var.grp.idx = var.qjnt.grp.idx.h0,
                                               subsidy.total = subsidy.total.default.h0)
list.subsidy.y.params.maximand.default.h0 <- list(vec.subsidy.frac =
                                               round(f_subsidy_frac(sca.subsidy.groups.h0, 'unif', seed=13), 3))

# Overall
list.subsidy.y.params.h0 <- append(list.subsidy.y.params.other.default.h0,
                                   list.subsidy.y.params.maximand.default.h0)

### For h0wlt

In [151]:
# Default Parameters for Estimation Set up
f.subsidy.y.str.h0wlt <- 'f_subsidy_y_cd'
t.filter.default.h0wlt <- 24
subsidy.total.default.h0wlt <- 100

df.default.h0wlt <- df.h0wlt %>% filter(t == t.filter.default.h0wlt)
list.subsidy.y.params.other.default.h0wlt <- list(df = df.default.h0wlt, 
                                                  res.linfe = res.linfe.h0wlt,
                                                  var.grp.idx = var.qjnt.grp.idx.h0wlt,
                                                  subsidy.total = subsidy.total.default.h0wlt)
list.subsidy.y.params.maximand.default.h0wlt <- list(vec.subsidy.frac = 
                                                     round(f_subsidy_frac(sca.subsidy.groups.h0wlt, 'unif', seed=13), 3))

# Overall
list.subsidy.y.params.h0wlt <- append(list.subsidy.y.params.other.default.h0wlt,
                                      list.subsidy.y.params.maximand.default.h0wlt)

### Testing Prediction Function

In [152]:
# Overall Parameter vector
options(repr.matrix.max.rows=5, repr.matrix.max.cols=20)
do.call(f.subsidy.y.str.i4xy, list.subsidy.y.params.i4xy)

i,t,y,x,grp.idx.i4xy,subsidy_grp,subsidy,y_subsidy
4,24,78.1,10.3,1,"0.528,0.183,0.289",52.8,97.10188
13,24,78.7,11.0,2,"0.528,0.183,0.289",18.3,88.06954
15,24,85.0,20.3,3,"0.528,0.183,0.289",28.9,93.34157


In [153]:
options(repr.matrix.max.rows=5, repr.matrix.max.cols=20)
do.call(f.subsidy.y.str.h0, list.subsidy.y.params.h0)

i,t,y,x,h0,w,h0_Qs0e1n2,cebu.grp.idx.h0,subsidy_grp,subsidy,y_subsidy
2,24,79.2,14.1,49.7,7.3,"(49.4,54]; (2) of Qs0e1n2",2,"0.5,0.5",50,88.39223
3,24,76.7,20.6,51.7,10.3,"(49.4,54]; (2) of Qs0e1n2",2,"0.5,0.5",50,80.47004
4,24,78.1,10.3,50.2,13.3,"(49.4,54]; (2) of Qs0e1n2",2,"0.5,0.5",50,90.81052
...,...,...,...,...,...,...,...,...,...,...
58,24,80.3,8.7,46.7,10.3,"[44.2,49.4]; (1) of Qs0e1n2",1,"0.5,0.5",50,80.22327
60,24,79.2,6.9,49.4,7.3,"[44.2,49.4]; (1) of Qs0e1n2",1,"0.5,0.5",50,85.20537


In [154]:
options(repr.matrix.max.rows=5, repr.matrix.max.cols=20)
do.call(f.subsidy.y.str.h0wlt, list.subsidy.y.params.h0wlt)

i,t,y,x,h0,w,h0_Qs0e1n2,w_Qs0e1n2,guat.grp.idx.h0wlt,subsidy_grp,subsidy,y_subsidy
1352,24,75.8,46.3,47.4,3.3,"[40.6,49.9]; (1) of Qs0e1n2","(2.2,5.7]; (2) of Qs0e1n2",2,"0.25,0.25,0.25,0.25",25,76.87116
1354,24,75.3,15.4,51.2,1.7,"(49.9,56.7]; (2) of Qs0e1n2","[1,2.2]; (1) of Qs0e1n2",3,"0.25,0.25,0.25,0.25",25,77.01000
1356,24,77.1,30.5,51.9,3.4,"(49.9,56.7]; (2) of Qs0e1n2","(2.2,5.7]; (2) of Qs0e1n2",4,"0.25,0.25,0.25,0.25",25,77.29038
...,...,...,...,...,...,...,...,...,...,...,...
2018,24,77.2,31.7,48.8,2.1,"[40.6,49.9]; (1) of Qs0e1n2","[1,2.2]; (1) of Qs0e1n2",1,"0.25,0.25,0.25,0.25",25,76.37334
2019,24,76.8,57.8,49.7,2.2,"[40.6,49.9]; (1) of Qs0e1n2","[1,2.2]; (1) of Qs0e1n2",1,"0.25,0.25,0.25,0.25",25,73.48729


## Planer Optimization Single CES

Single CES Planer Optimization problem. 

In [155]:
options(warn=-1) # Suppress Warning Messages
# optim_wrapper() from C:\Users\fan\R4Econ\optimization\planer\ces\cesoptimizer.R
param.ces <- 1
# test i4xy
res.opti.test.i4xy <- optim_wrapper(sca.subsidy.frac.init = (numeric((sca.subsidy.groups.i4xy-1))+1), 
                                    param.ces = param.ces, f.subsidy.y.str = f.subsidy.y.str.i4xy,
                                    list.subsidy.y.params.other = list.subsidy.y.params.other.default.i4xy)
# test h0
res.opti.test.h0 <- optim_wrapper(sca.subsidy.frac.init = (numeric((sca.subsidy.groups.h0-1))+1), 
                                  param.ces = param.ces, f.subsidy.y.str = f.subsidy.y.str.h0,
                                  list.subsidy.y.params.other = list.subsidy.y.params.other.default.h0)
# test h0wlt
res.opti.test.h0wlt <- optim_wrapper(sca.subsidy.frac.init = (numeric((sca.subsidy.groups.h0wlt-1))+1), 
                                     param.ces = param.ces, f.subsidy.y.str = f.subsidy.y.str.h0wlt,
                                     list.subsidy.y.params.other = list.subsidy.y.params.other.default.h0wlt)

In [156]:
# Display Test Results
options(repr.matrix.max.rows=20, repr.matrix.max.cols=20)
df.test.results <- list(df_i4xy=res.opti.test.i4xy,
                        df_h0=res.opti.test.h0, 
                        df_h0wlt=res.opti.test.h0wlt)
# Show Results From Tests
# list.names = list of list first level names
# list.of.list.names = list of list second level names
as.data.frame(df.test.results) %>%
    gather(variable, value) %>%
    separate(variable, c('list.names', 'list.of.list.names'),
             sep = "\\.", extra = "merge") %>%
    spread(list.names, value) %>%
    column_to_rownames(var='list.of.list.names')

,df_h0,df_h0wlt,df_i4xy
convergence,0.0000000,0.0000000,0.0000000
counts.function,20.0000000,108.0000000,55.0000000
counts.gradient,NA,NA,NA
par.frac.sum,1.0000000,1.0000000,1.0000000
par.frac.v1,0.5109358,0.3599645,0.3740215
par.frac.v2,0.4890642,0.2133391,0.3584842
par.frac.v3,NA,0.1533452,0.2674943
par.frac.v4,NA,0.2733512,NA
par.v1,0.0437500,-0.5755182,-0.5150028
par.v2,NA,-0.6931901,0.2927862
